In [21]:
from collections import defaultdict
from tqdm import tqdm
import pandas as pd

In [24]:
orf_id_to_seqs = defaultdict(list)
seq_to_orf_ids = defaultdict(list)
read_id_to_seqs = defaultdict(list)
rows = []
curr_id = None
curr_lines = []
def add_line():
    global curr_id
    global curr_lines
    if curr_id:
        prot = "".join(curr_lines).replace("*", "")
        seq_to_orf_ids[prot].append(curr_id)
        
        orf_id_to_seqs[curr_id].append(prot)
        read_id = "_".join(curr_id.split("_")[2:])
        read_id_to_seqs[read_id].append(prot)
        seq_to_read_ids[prot].append(read_id)
        rows.append({"seq": prot, "ORF": curr_id, "read_id": read_id})
    curr_lines = []
    curr_id = None      
with open("data/translated_reads_v2.fa") as f:
    for l in tqdm(f.read().splitlines()):
        if l.startswith(">"):
            add_line()    
            curr_id = l[1:].strip()
        else:
            l = l.strip()
            if l:
                curr_lines.append(l)
            else:
                print("WARNING: blank line")
add_line()
df_seqs = pd.DataFrame(rows)
print("%d unique protein seqs, %d read IDs, %d total entries" % (len(seq_to_read_ids), len(read_id_to_seqs), len(orf_id_to_seqs)))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364208/364208 [00:00<00:00, 813005.47it/s]

77893 unique protein seqs, 27595 read IDs, 137472 total entries


In [26]:
peptide_to_source_seqs = defaultdict(set)
peptide_to_source_read_ids = defaultdict(set)
peptide_to_source_orf_ids = defaultdict(set)

for seq, orf_ids in tqdm(list(seq_to_orf_ids.items())):
    n = len(seq)
    read_ids = [
        "_".join(orf_id.split("_")[2:])
        for orf_id in orf_ids
    ]
    
    for k in [8, 9, 10, 11]:
        for i in range(n - k + 1):
            p = seq[i:i + k]
            peptide_to_source_seqs[p].add(seq)
            peptide_to_source_orf_ids[p].update(orf_ids)
            peptide_to_source_read_ids[p].update(read_ids)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77893/77893 [03:45<00:00, 345.67it/s]


In [27]:
peptide_rows = []
for p, source_seqs in tqdm(list(peptide_to_source_seqs.items())):
    peptide_rows.append({
        "peptide": p, 
        "peptide_len": len(p), 
        "num_source_seqs": ";".join(sorted(source_seqs)), 
        "num_source_orfs": ";".join(sorted(peptide_to_source_orf_ids[p])),
        "num_source_reads": ";".join(sorted(peptide_to_source_read_ids[p])),
    })
df_peptides = pd.DataFrame(peptide_rows)
df_peptides

  0%|                                                                                                                                                                      | 7009/15329165 [01:29<9:54:29, 429.56it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x111519250>>
Traceback (most recent call last):
  File "/Users/iskander/code/shared-virtual-env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15329165/15329165 [02:09<00:00, 118562.79it/s]


,peptide,peptide_len,source_seqs,source_orfs,source_reads
0,MCSGPGPG,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...
1,CSGPGPGS,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...
2,SGPGPGSF,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...
3,GPGPGSFA,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...
4,PGPGSFAT,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...
...,...,...,...,...,...
15329160,DNVILLITAHC,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1
15329161,NVILLITAHCT,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1
15329162,VILLITAHCTS,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1
15329163,ILLITAHCTSV,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1


In [31]:
df_peptides["num_source_seqs"] = df_peptides["source_seqs"].str.count(';') + 1
df_peptides["num_source_orfs"] = df_peptides["source_orfs"].str.count(';') + 1
df_peptides["num_source_reads"] = df_peptides["source_reads"].str.count(';') + 1
df_peptides.to_csv("peptides-8mer-to-11mer-with-source.csv")
df_peptides

,peptide,peptide_len,source_seqs,source_orfs,source_reads,num_source_seqs,num_source_orfs,num_source_reads
0,MCSGPGPG,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
1,CSGPGPGS,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
2,SGPGPGSF,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
3,GPGPGSFA,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
4,PGPGSFAT,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
...,...,...,...,...,...,...,...,...
15329160,DNVILLITAHC,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1
15329161,NVILLITAHCT,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1
15329162,VILLITAHCTS,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1
15329163,ILLITAHCTSV,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1


In [33]:
import mhcflurry 
pred = mhcflurry.Class1PresentationPredictor().load()
df_pred = pred.predict(list(df_peptides.peptide), alleles=["H2-Kb", "H2-Db"])

Predicting processing.


  0%|                                                                                                                                                                                         | 0/154 [00:00<?, ?it/s]

25/25 [==============================] - 1s 47ms/step


  1%|█▏                                                                                                                                                                               | 1/154 [00:11<29:58, 11.75s/it]

25/25 [==============================] - 1s 48ms/step


...

  0%|                                                                                                                                          | 0/2 [00:00<?, ?it/s]/Users/iskander/code/shared-virtual-env/lib/python3.12/site-packages/mhcflurry/amino_acid.py:132: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = df.replace(letter_to_index_dict)


3743/3743 [==============================] - 916s 245ms/step


 50%|████████████████████████████████████████████████████████████████                                                                | 1/2 [16:41<16:41, 1001.56s/it]/Users/iskander/code/shared-virtual-env/lib/python3.12/site-packages/mhcflurry/amino_acid.py:132: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = df.replace(letter_to_index_dict)


3743/3743 [==============================] - 905s 242ms/step


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [32:59<00:00, 989.73s/it]


In [34]:
df_pred

,peptide,peptide_num,sample_name,affinity,best_allele,processing_score,presentation_score,presentation_percentile
0,MCSGPGPG,0,sample1,27238.952145,H2-Kb,0.000256,0.003703,99.286603
1,CSGPGPGS,1,sample1,24759.293829,H2-Kb,0.000350,0.004062,62.744674
2,SGPGPGSF,2,sample1,4000.852173,H2-Kb,0.076324,0.030392,8.015652
3,GPGPGSFA,3,sample1,25551.819781,H2-Kb,0.007762,0.004044,62.744674
4,PGPGSFAT,4,sample1,25944.723804,H2-Kb,0.004395,0.003938,99.286603
...,...,...,...,...,...,...,...,...
15329160,DNVILLITAHC,15329160,sample1,26815.714720,H2-Kb,0.003451,0.003802,99.286603
15329161,NVILLITAHCT,15329161,sample1,28573.494144,H2-Kb,0.006022,0.003608,99.286603
15329162,VILLITAHCTS,15329162,sample1,28408.690877,H2-Kb,0.002364,0.003581,99.286603
15329163,ILLITAHCTSV,15329163,sample1,22988.379779,H2-Kb,0.017955,0.004644,62.744674


In [35]:
df_peptides

,peptide,peptide_len,source_seqs,source_orfs,source_reads,num_source_seqs,num_source_orfs,num_source_reads
0,MCSGPGPG,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
1,CSGPGPGS,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
2,SGPGPGSF,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
3,GPGPGSFA,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
4,PGPGSFAT,8,MCSGPGPGSFATVAGDAKPRARSGLGQSFECVMASSSDSEDDSVMA...,ORF_645387_m84165_240529_114357_s1/100008387/c...,m84165_240529_114357_s1/100008387/ccs/6720_118...,1,1,1
...,...,...,...,...,...,...,...,...
15329160,DNVILLITAHC,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1
15329161,NVILLITAHCT,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1
15329162,VILLITAHCTS,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1
15329163,ILLITAHCTSV,11,MQRELSAAQGSPLLAATAAPPATLGGDGDRQRGRSDYERLESTPAA...,ORF_24_m84165_240529_114357_s1/99947067/ccs/27...,m84165_240529_114357_s1/99947067/ccs/2778_3720_1,1,1,1


In [36]:
df_merge = pd.merge(df_peptides, df_pred, left_on='peptide', right_on='peptide')

In [ ]:
df_merge.to_csv("peptides-with-predictions.csv")

In [ ]:
df_top = df_merge[df_merge.presentation_percentile < 1]
df_top 